In [1]:
import json
import sys
from urllib.parse import urlparse, parse_qs, urljoin

import requests as rq
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd
import os
import re

dir_path = "C:\\Users\\user\\Desktop\\python\\datamining\\컷툰_작가_시간_19세\\" ## 수정부분

from selenium import webdriver 

In [2]:
NAVER_URL = "https://comic.naver.com/index.nhn"
TOP_URL = "https://comic.naver.com/webtoon/weekday.nhn"

In [3]:
def save(data, file_name):
    file = open(file_name, 'a')
    file.write(data + '\n')

In [2]:
def get_daylywebtoons():
    '''
    요일 웹툰을 수집
    '''

    webtoon_main_url = "TOP_URL"

    res = rq.get(webtoon_main_url)

    main_soup = BeautifulSoup(res.content, 'lxml')

    webtoon_links = [{"title": a_tag.get('title'), "link": urljoin(NAVER_URL, a_tag.get('href'))}
                      for a_tag in main_soup.select('.daily_all a.title')]

    return webtoon_links


In [3]:
def make_link(webtoon_url, page_count):
    return webtoon_url + '&page=' + str(page_count)


def get_qs(url, key):
    url_query = urlparse(url).query
    query_dict = parse_qs(url_query)
    value = query_dict[key][0]
    return value

In [4]:

def get_all_webtoon(webtoon, is_save):
    '''
    해당 웹 툰의  1화 ~ 마지막까지 수집
    '''
    page_count = 1

    target_webtoons = list()
    webtoon_url = webtoon['link']
    webtoon_title = webtoon['title']

    webtoon_id = get_qs(webtoon_url, 'titleId')
    weekday = get_qs(webtoon_url, 'weekday')

    is_unlast = True

    while is_unlast:
        link = make_link(webtoon_url, page_count)

        target_webtoon_res = rq.get(link)
        webtoon_soup = BeautifulSoup(target_webtoon_res.content, 'lxml')
        a_tags = webtoon_soup.select('.viewList td.title a')

        for a_tag in a_tags:
            t = a_tag.text.replace('\n', '').replace('\r', '').replace('\t', '')
            h = urljoin(NAVER_URL, a_tag.get('href'))

            if h not in target_webtoons:
                target_webtoons.append(h)
            else:
                is_unlast = False

        page_count += 1

    if is_save:
        for webtoon in target_webtoons:
            save(webtoon_title + ':' + webtoon, 'all_webtoons.txt')

    return webtoon_title.strip(), webtoon_id, weekday, target_webtoons



In [5]:
def data_parse2(soup, url):
    title = soup.title.text.split(':')[0].lstrip(" ").rstrip(" ")
    no_title = soup.select('.tit_area .view h3')[0].text
    no_title = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』姊\\‘|\(\)\[\]\<\>`\'… 【】▲▶◇ⓒⓒ■~★○●◆♥》└ ┘]', '', no_title)
    no_title = re.sub('下', '하', no_title)
    no_title = re.sub('上', '상', no_title)
    no_title = re.sub("\u200b","",no_title)
    
    no_date = soup.find("dd",{"class":"date"}).text
    no_time = soup.find("input",{"name":"itemDt"}).get("value").split(" ")[1]
  
    num_cut = soup.find("div",{"class":"wt_viewer"}).find_all("img",{"alt":"comic content"})
    writer_info = soup.find("div",{"class":"writer_info"}).find("p").text
    
    check=0
    if len(writer_info) != 1 : 
        check +=1
        
    cut_writer = [title,no_title,no_date,no_time,len(num_cut),check]
 
    return(cut_writer)

In [6]:
import time
driver = webdriver.Chrome("C:\\Users\\user\\chromedriver_win32\\chromedriver.exe")
driver.get('https://nid.naver.com/nidlogin.login')

id = 'abcd11094'  #ID  직접 입력
pw = 'skfkdrkwk0723!' #비번은 직접 입력
driver.execute_script("document.getElementsByName('id')[0].value=\'" + id + "\'")
   # time.sleep(1)
driver.execute_script("document.getElementsByName('pw')[0].value=\'" + pw + "\'")
    # time.sleep(1)

driver.find_element_by_xpath('//*[@id="frmNIDLogin"]/fieldset/input').click()
time.sleep(0.005)

#이것도 
#27,184
if __name__ == "__main__":
    webtoons = get_daylywebtoons()
    webtoon_title, title_id, weekday, target_webtoons = get_all_webtoon(webtoons[133], False)## 수정부분
    os.mkdir(dir_path + webtoon_title)
    total =[]
    for webtoon_page in target_webtoons:
        driver.get(webtoon_page)
        html = driver.page_source
        webtoon_page_soup = BeautifulSoup(html, 'lxml')
        part=data_parse2(webtoon_page_soup, webtoon_page)
        sleep(0.005)
        total.append(part)
    txttotal = pd.DataFrame(total)
    txttotal.to_csv(dir_path+webtoon_title+"\\"+"cut_writer"+'.csv', index=False, header=False,encoding="cp949") 

NameError: name 'webdriver' is not defined

In [14]:
import time
driver = webdriver.Chrome("C:\\Users\\user\\chromedriver_win32\\chromedriver.exe")
driver.get('https://nid.naver.com/nidlogin.login')

id = 'abcd11094'  #ID  직접 입력
pw = 'sp0723tygkfkrh' #비번은 직접 입력
driver.execute_script("document.getElementsByName('id')[0].value=\'" + id + "\'")
   # time.sleep(1)
driver.execute_script("document.getElementsByName('pw')[0].value=\'" + pw + "\'")
    # time.sleep(1)

driver.find_element_by_xpath('//*[@id="frmNIDLogin"]/fieldset/input').click()
time.sleep(0.005)

#이것도 
#27,184
if __name__ == "__main__":
    webtoons = get_daylywebtoons()
    webtoon_title, title_id, weekday, target_webtoons = get_all_webtoon(webtoons[59], False)## 수정부분
    os.mkdir(dir_path + webtoon_title)
    total =[]
    for webtoon_page in target_webtoons:
        driver.get(webtoon_page)
        html = driver.page_source
        webtoon_page_soup = BeautifulSoup(html, 'lxml')
        part=data_parse2(webtoon_page_soup, webtoon_page)
        sleep(0.005)
        total.append(part)
    txttotal = pd.DataFrame(total)
    txttotal.to_csv(dir_path+webtoon_title+"\\"+"cut_writer"+'.csv', index=False, header=False,encoding="cp949") 

In [15]:
import time
driver = webdriver.Chrome("C:\\Users\\user\\chromedriver_win32\\chromedriver.exe")
driver.get('https://nid.naver.com/nidlogin.login')

id = 'abcd11094'  #ID  직접 입력
pw = 'sp0723tygkfkrh' #비번은 직접 입력
driver.execute_script("document.getElementsByName('id')[0].value=\'" + id + "\'")
   # time.sleep(1)
driver.execute_script("document.getElementsByName('pw')[0].value=\'" + pw + "\'")
    # time.sleep(1)

driver.find_element_by_xpath('//*[@id="frmNIDLogin"]/fieldset/input').click()
time.sleep(0.005)

#이것도 
#27,184
if __name__ == "__main__":
    webtoons = get_daylywebtoons()
    webtoon_title, title_id, weekday, target_webtoons = get_all_webtoon(webtoons[66], False)## 수정부분
    os.mkdir(dir_path + webtoon_title)
    total =[]
    for webtoon_page in target_webtoons:
        driver.get(webtoon_page)
        html = driver.page_source
        webtoon_page_soup = BeautifulSoup(html, 'lxml')
        part=data_parse2(webtoon_page_soup, webtoon_page)
        sleep(0.005)
        total.append(part)
    txttotal = pd.DataFrame(total)
    txttotal.to_csv(dir_path+webtoon_title+"\\"+"cut_writer"+'.csv', index=False, header=False,encoding="cp949") 

In [21]:
import time
driver = webdriver.Chrome("C:\\Users\\user\\chromedriver_win32\\chromedriver.exe")
driver.get('https://nid.naver.com/nidlogin.login')

id = 'abcd11094'  #ID  직접 입력
pw = 'sp0723tygkfkrh' #비번은 직접 입력
driver.execute_script("document.getElementsByName('id')[0].value=\'" + id + "\'")
   # time.sleep(1)
driver.execute_script("document.getElementsByName('pw')[0].value=\'" + pw + "\'")
    # time.sleep(1)

driver.find_element_by_xpath('//*[@id="frmNIDLogin"]/fieldset/input').click()
time.sleep(0.005)

#이것도 
#27,184
if __name__ == "__main__":
    webtoons = get_daylywebtoons()
    webtoon_title, title_id, weekday, target_webtoons = get_all_webtoon(webtoons[77], False)## 수정부분
   # os.mkdir(dir_path + webtoon_title)
    total =[]
    for webtoon_page in target_webtoons:
        driver.get(webtoon_page)
        html = driver.page_source
        webtoon_page_soup = BeautifulSoup(html, 'lxml')
        part=data_parse2(webtoon_page_soup, webtoon_page)
        sleep(0.005)
        total.append(part)
    txttotal = pd.DataFrame(total)
    txttotal.to_csv(dir_path+"헬퍼 2"+"\\"+"cut_writer"+'.csv', index=False, header=False,encoding="cp949") 

In [17]:
import time
driver = webdriver.Chrome("C:\\Users\\user\\chromedriver_win32\\chromedriver.exe")
driver.get('https://nid.naver.com/nidlogin.login')

id = 'abcd11094'  #ID  직접 입력
pw = 'sp0723tygkfkrh' #비번은 직접 입력
driver.execute_script("document.getElementsByName('id')[0].value=\'" + id + "\'")
   # time.sleep(1)
driver.execute_script("document.getElementsByName('pw')[0].value=\'" + pw + "\'")
    # time.sleep(1)

driver.find_element_by_xpath('//*[@id="frmNIDLogin"]/fieldset/input').click()
time.sleep(0.005)

#이것도 
#27,184
if __name__ == "__main__":
    webtoons = get_daylywebtoons()
    webtoon_title, title_id, weekday, target_webtoons = get_all_webtoon(webtoons[130], False)## 수정부분
    os.mkdir(dir_path + webtoon_title)
    total =[]
    for webtoon_page in target_webtoons:
        driver.get(webtoon_page)
        html = driver.page_source
        webtoon_page_soup = BeautifulSoup(html, 'lxml')
        part=data_parse2(webtoon_page_soup, webtoon_page)
        sleep(0.005)
        total.append(part)
    txttotal = pd.DataFrame(total)
    txttotal.to_csv(dir_path+webtoon_title+"\\"+"cut_writer"+'.csv', index=False, header=False,encoding="cp949") 

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=74.0.3729.131)
  (Driver info: chromedriver=2.40.565498 (ea082db3280dd6843ebfb08a625e3eb905c4f5ab),platform=Windows NT 10.0.17134 x86_64)


In [20]:
import time
driver = webdriver.Chrome("C:\\Users\\user\\chromedriver_win32\\chromedriver.exe")
driver.get('https://nid.naver.com/nidlogin.login')

id = 'abcd11094'  #ID  직접 입력
pw = 'sp0723tygkfkrh' #비번은 직접 입력
driver.execute_script("document.getElementsByName('id')[0].value=\'" + id + "\'")
   # time.sleep(1)
driver.execute_script("document.getElementsByName('pw')[0].value=\'" + pw + "\'")
    # time.sleep(1)

driver.find_element_by_xpath('//*[@id="frmNIDLogin"]/fieldset/input').click()
time.sleep(0.005)

#이것도 
#27,184
if __name__ == "__main__":
    webtoons = get_daylywebtoons()
    webtoon_title, title_id, weekday, target_webtoons = get_all_webtoon(webtoons[183], False)## 수정부분
    os.mkdir(dir_path + webtoon_title)
    total =[]
    for webtoon_page in target_webtoons:
        driver.get(webtoon_page)
        html = driver.page_source
        webtoon_page_soup = BeautifulSoup(html, 'lxml')
        part=data_parse2(webtoon_page_soup, webtoon_page)
        sleep(0.005)
        total.append(part)
    txttotal = pd.DataFrame(total)
    txttotal.to_csv(dir_path+webtoon_title+"\\"+"cut_writer"+'.csv', index=False, header=False,encoding="cp949") 

In [19]:
import time
driver = webdriver.Chrome("C:\\Users\\user\\chromedriver_win32\\chromedriver.exe")
driver.get('https://nid.naver.com/nidlogin.login')

id = 'abcd11094'  #ID  직접 입력
pw = 'sp0723tygkfkrh' #비번은 직접 입력
driver.execute_script("document.getElementsByName('id')[0].value=\'" + id + "\'")
   # time.sleep(1)
driver.execute_script("document.getElementsByName('pw')[0].value=\'" + pw + "\'")
    # time.sleep(1)

driver.find_element_by_xpath('//*[@id="frmNIDLogin"]/fieldset/input').click()
time.sleep(0.005)

#이것도 
#27,184
if __name__ == "__main__":
    webtoons = get_daylywebtoons()
    webtoon_title, title_id, weekday, target_webtoons = get_all_webtoon(webtoons[203], False)## 수정부분
    os.mkdir(dir_path + webtoon_title)
    total =[]
    for webtoon_page in target_webtoons:
        driver.get(webtoon_page)
        html = driver.page_source
        webtoon_page_soup = BeautifulSoup(html, 'lxml')
        part=data_parse2(webtoon_page_soup, webtoon_page)
        sleep(0.005)
        total.append(part)
    txttotal = pd.DataFrame(total)
    txttotal.to_csv(dir_path+webtoon_title+"\\"+"cut_writer"+'.csv', index=False, header=False,encoding="cp949") 